In [27]:
%load_ext autoreload
%autoreload 2

import copy 
import sys
from pathlib import Path

sys.path.append('./')
sys.path.append('../')
sys.path.append('../..')

def show( object ):
  return [ x for x in dir( object ) if not x.startswith('__') ] 

    
import pprint 
import pandas as pd
#print( pd.__version__)  # must be 1.3.5 

import numpy as np
#print( np.__version__)  # must be 1.23.5

assert( str(pd.__version__) == '1.3.5')
assert( str(np.__version__) == '1.23.5')

from wf_lib2.crm_definitions import *
from wf_lib2.data.crm_dataset  import CRMDataset
from wf_lib2.data.crm_pattern  import CRMPattern
from wf_lib2.view_model.view_model import * 
import plotly.io as pio

print('Methods and properties of the CRMDataset class')
show( CRMDataset )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Methods and properties of the CRMDataset class


['_generate_dataset',
 '_get_distance_patterns_helper',
 '_slice_dates_dataset',
 'check_dataset',
 'copy_from',
 'filter_by',
 'from_pattern',
 'generate_default_dataset',
 'generate_default_multiwell_dataset',
 'get_column_names',
 'get_distance_patterns',
 'get_distances',
 'get_distances_flat',
 'get_explicit_pattern',
 'get_pattern',
 'injector_locations',
 'injector_names',
 'instance',
 'mode_sampling_frequency',
 'producer_locations',
 'producer_names',
 'slice_coordinates_dataset',
 'slice_dates_dataset',
 'very_old_get_distance_patterns',
 'very_old_get_distance_patterns_helper']

In [28]:
dataset = CRMDataset.generate_default_multiwell_dataset() 
display(dataset.injectors_df.head(5))
display(dataset.producers_df.head(5))
display(dataset.locations_df)

w = dataset.producers_df.copy()
w['GAS_VOLUME'] = 0.1 * w['LIQUID_VOLUME']
w['OIL_VOLUME'] = 0.2 * w['LIQUID_VOLUME']
w['WATER_VOLUME'] = 0.1 * w['LIQUID_VOLUME']
dataset.producers_df = w 


1053 1053
Tying to guess the DATE format. dayforst ? False


,DATE,NAME,WATER_INJECTION_VOLUME,ZONE,SUBZONE,SECTOR,RESERVOIR
0,2022-01-23,Inj0,354.719228,Unique,Unique,0,Unique
1,2022-01-24,Inj0,354.716843,Unique,Unique,0,Unique
2,2022-01-25,Inj0,354.720018,Unique,Unique,0,Unique
3,2022-01-26,Inj0,354.720355,Unique,Unique,0,Unique
4,2022-01-27,Inj0,294.735906,Unique,Unique,0,Unique


,DATE,NAME,LIQUID_VOLUME,WATER_VOLUME,GAS_VOLUME,OIL_VOLUME,ZONE,SUBZONE,SECTOR,RESERVOIR
0,2022-01-23,Producer1,30.598382,30.598382,0.0,0.0,Unique,Unique,0,Unique
1,2022-01-24,Producer1,59.143985,59.143985,0.0,0.0,Unique,Unique,0,Unique
2,2022-01-25,Producer1,85.848004,85.848004,0.0,0.0,Unique,Unique,0,Unique
3,2022-01-26,Producer1,110.830344,110.830344,0.0,0.0,Unique,Unique,0,Unique
4,2022-01-27,Producer1,131.879565,131.879565,0.0,0.0,Unique,Unique,0,Unique


,NAME,X,Y,ZONE,SUBZONE,SECTOR,RESERVOIR,TYPE
0,Inj0,1000,0,Unique,Unique,0,Unique,Injector
1,Inj1,0,1000,Unique,Unique,0,Unique,Injector
2,Inj2,-1000,0,Unique,Unique,0,Unique,Injector
3,Inj3,0,-1000,Unique,Unique,0,Unique,Injector
4,Producer1,0,0,Unique,Unique,0,Unique,Producer
5,Producer2,500,500,Unique,Unique,0,Unique,Producer
6,Producer3,1000,1000,Unique,Unique,0,Unique,Producer


In [91]:
filters = {
    'zone': 'Unique', 'subzone':'Unique'
}
d= apply_dataset_filters( dataset, filters )
p = d.get_pattern(fix_time_gaps=True, fill_nan=0.0)

def fractions_plot( pattern, cummulative = False ):
    liquid_production_summary = pattern.liquid_production.sum(axis=1)
    water_production_summary  = pattern.water_production.sum(axis=1)
    oil_production_summary    = pattern.oil_production.sum(axis=1)
    gas_production_summary    = pattern.gas_production.sum(axis=1)
    liquid_production_summary
    
    fractions_dfs = [water_production_summary, oil_production_summary,gas_production_summary]
    fractions_labels = ['Water', 'Oil','Gas']
    fractions_data   = []
    sum_values = 1.0 
    last_date = water_production_summary.index.max() 

    for n, w in enumerate(fractions_dfs):
        if cummulative:
            w = w.cumsum(axis = 0 )  
            
        last_date_value = w.loc[ w.index.max() ]
        fractions_data.append(last_date_value)
        sum_values = sum_values + last_date_value 
        last_date = max( last_date, w.index.max() )
    for n in range( len(fractions_data )):
        fractions_data[n] = round( fractions_data[n] / sum_values, 1 ) 
    
        x = 'cummulative' if cummulative == True else '' 
        layout = copy.deepcopy( default_plotly_layout_python() )
        title = f'Volume fractions {x} present-day ({last_date.strftime("%d-%m-%Y")})'
        fig_dict  = {'data' : { 'values':fractions_data,'labels':fractions_labels, 'type':'pie','hole':0.4, 'name':'Fractions'}}
        fig_dict['layout'] = {'title':title}
        fig_dict['layout']['legend'] =dict(orientation='h',yanchor='bottom',y=-0.2,xanchor='center',x=0.5)
    return fig_dict

fig = fractions_plot( p, cummulative = False )
pio.show( fig )


We have a fix-time gaps instruction
